# Looking for the best Formula 1 season

For my master's project, I'm making a piece about answering the question: **What championship winning team had the best Formula 1 season?**

To do this I was working with data provided by the [Ergast Developer API](https://ergast.com/mrd/). I noticed an error in the driver-constructor pairing for the 1950 season and wanted to verify things without moving forward. I was originally going to create a table of the driver-constructor pairs for each race, and then compare it with the data I had.

Instead I've chosen to go straight to the source for F1 information ([formula1.com](https://formula1.com)) and scrape the race results for each race. I did this scraping on 2019-06-21 and 2019-06-22 and I'll be now working with that data to do my analysis.

Because it is data from a primary-source, I have some more confidence in it.

In [1]:
import pandas as pd
import numpy as np

The first thing to do is to import the data

In [2]:
race_results = pd.read_csv("../formula1-data/results_all_updated.csv")

In [3]:
race_results.head()

,raceId,year,raceRound,date,prix,driverFirstName,driverLastName,driverCode,constructor,finishingPosition,positionOrder,laps,time,points
0,1,1950,1,13 May 1950,Great Britain,Nino,Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,9.0
1,1,1950,1,13 May 1950,Great Britain,Luigi,Fagioli,FAG,Alfa Romeo,2,2,70.0,+2.600s,6.0
2,1,1950,1,13 May 1950,Great Britain,Reg,Parnell,PAR,Alfa Romeo,3,3,70.0,+52.000s,4.0
3,1,1950,1,13 May 1950,Great Britain,Yves Giraud,Cabantous,CAB,Talbot-Lago,4,4,68.0,+2 laps,3.0
4,1,1950,1,13 May 1950,Great Britain,Louis,Rosier,ROS,Talbot-Lago,5,5,68.0,+2 laps,2.0


In [4]:
race_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22535 entries, 0 to 22534
Data columns (total 14 columns):
raceId               22535 non-null int64
year                 22535 non-null int64
raceRound            22535 non-null int64
date                 22535 non-null object
prix                 22535 non-null object
driverFirstName      22535 non-null object
driverLastName       22535 non-null object
driverCode           22535 non-null object
constructor          22509 non-null object
finishingPosition    22535 non-null object
positionOrder        22535 non-null int64
laps                 22311 non-null float64
time                 22527 non-null object
points               22535 non-null float64
dtypes: float64(2), int64(4), object(8)
memory usage: 2.4+ MB


Let's also check how many races we have:

In [5]:
race_results.raceId.max()

1004

Things seem to be in good order. For most of the work, I won't particularly care for a few of the columns, namely:

* laps
* points
* driverCode
* date

So, let's drop those:

In [6]:
results = race_results.copy().drop(columns=["date", "laps", "points", "driverCode"])
results = results[results.year < 2019]

In [7]:
results.head()

,raceId,year,raceRound,prix,driverFirstName,driverLastName,constructor,finishingPosition,positionOrder,time
0,1,1950,1,Great Britain,Nino,Farina,Alfa Romeo,1,1,2:13:23.600
1,1,1950,1,Great Britain,Luigi,Fagioli,Alfa Romeo,2,2,+2.600s
2,1,1950,1,Great Britain,Reg,Parnell,Alfa Romeo,3,3,+52.000s
3,1,1950,1,Great Britain,Yves Giraud,Cabantous,Talbot-Lago,4,4,+2 laps
4,1,1950,1,Great Britain,Louis,Rosier,Talbot-Lago,5,5,+2 laps


For most of my analysis, I'm looking only at the teams that won championships, so let's slice the results table and keep only the different championship runs. I scraped more of the F1 site to find the teams that the winning drivers were a part of each season and am working with that.

In [8]:
teams = pd.read_csv("../formula1-data/winners_clean.csv")

In [9]:
teams

,year,constructor,constructor_clean
0,1950,Alfa Romeo,Alfa Romeo
1,1951,Alfa Romeo,Alfa Romeo
2,1952,Ferrari,Ferrari
3,1953,Ferrari,Ferrari
4,1954,Mercedes-Benz,Mercedes
5,1955,Mercedes-Benz,Mercedes
6,1956,Ferrari,Ferrari
7,1957,Maserati,Maserati
8,1958,Ferrari,Ferrari
9,1959,Cooper Climax,Cooper


One thing with the teams is that they change names of the year (look at McLaren for instance) and often their name is the constructor followed by the engine they're using during a particular year.

To account for this, I created a column of clean names that keep the constructor consistent across the years. To slice my results data, I will merge this table to the results table and only keep the lines that match both `year` and `constructor`.

In [10]:
comparison = pd.merge(results, teams, how="left", on=["year", "constructor"], indicator=True)

In [11]:
comparison.head()

,raceId,year,raceRound,prix,driverFirstName,driverLastName,constructor,finishingPosition,positionOrder,time,constructor_clean,_merge
0,1,1950,1,Great Britain,Nino,Farina,Alfa Romeo,1,1,2:13:23.600,Alfa Romeo,both
1,1,1950,1,Great Britain,Luigi,Fagioli,Alfa Romeo,2,2,+2.600s,Alfa Romeo,both
2,1,1950,1,Great Britain,Reg,Parnell,Alfa Romeo,3,3,+52.000s,Alfa Romeo,both
3,1,1950,1,Great Britain,Yves Giraud,Cabantous,Talbot-Lago,4,4,+2 laps,NaN,left_only
4,1,1950,1,Great Britain,Louis,Rosier,Talbot-Lago,5,5,+2 laps,NaN,left_only


In [12]:
results_slice = comparison[comparison._merge == "both"]
results_slice.head(22)

,raceId,year,raceRound,prix,driverFirstName,driverLastName,constructor,finishingPosition,positionOrder,time,constructor_clean,_merge
0,1,1950,1,Great Britain,Nino,Farina,Alfa Romeo,1,1,2:13:23.600,Alfa Romeo,both
1,1,1950,1,Great Britain,Luigi,Fagioli,Alfa Romeo,2,2,+2.600s,Alfa Romeo,both
2,1,1950,1,Great Britain,Reg,Parnell,Alfa Romeo,3,3,+52.000s,Alfa Romeo,both
12,1,1950,1,Great Britain,Juan Manuel,Fangio,Alfa Romeo,NC,13,DNF,Alfa Romeo,both
23,2,1950,2,Monaco,Juan Manuel,Fangio,Alfa Romeo,1,1,3:13:18.700,Alfa Romeo,both
33,2,1950,2,Monaco,Luigi,Fagioli,Alfa Romeo,NC,11,DNF,Alfa Romeo,both
34,2,1950,2,Monaco,Nino,Farina,Alfa Romeo,NC,12,DNF,Alfa Romeo,both
78,4,1950,4,Switzerland,Nino,Farina,Alfa Romeo,1,1,2:02:53.700,Alfa Romeo,both
79,4,1950,4,Switzerland,Luigi,Fagioli,Alfa Romeo,2,2,+0.400s,Alfa Romeo,both
89,4,1950,4,Switzerland,Juan Manuel,Fangio,Alfa Romeo,NC,12,DNF,Alfa Romeo,both


Now that I've done this, I can go in and drop the old constructor column and the newly added `_merge` column

In [13]:
results2 = results_slice.drop(columns=["constructor", "_merge"]) 

In [14]:
results2 = results2.rename(index=str, columns={"constructor_clean": "constructor"})

In [15]:
results2.head()

,raceId,year,raceRound,prix,driverFirstName,driverLastName,finishingPosition,positionOrder,time,constructor
0,1,1950,1,Great Britain,Nino,Farina,1,1,2:13:23.600,Alfa Romeo
1,1,1950,1,Great Britain,Luigi,Fagioli,2,2,+2.600s,Alfa Romeo
2,1,1950,1,Great Britain,Reg,Parnell,3,3,+52.000s,Alfa Romeo
12,1,1950,1,Great Britain,Juan Manuel,Fangio,NC,13,DNF,Alfa Romeo
23,2,1950,2,Monaco,Juan Manuel,Fangio,1,1,3:13:18.700,Alfa Romeo


In [16]:
results2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2293 entries, 0 to 22379
Data columns (total 10 columns):
raceId               2293 non-null int64
year                 2293 non-null int64
raceRound            2293 non-null int64
prix                 2293 non-null object
driverFirstName      2293 non-null object
driverLastName       2293 non-null object
finishingPosition    2293 non-null object
positionOrder        2293 non-null int64
time                 2293 non-null object
constructor          2293 non-null object
dtypes: int64(4), object(6)
memory usage: 197.1+ KB


Now we can head into the analysis.

---

## Idea 01: Wins

The first way we can look to find an answer to the question is to look at the wins each team had in their run.

In [17]:
wins = results2[results2.finishingPosition == "1"]

In [18]:
wins.head()

,raceId,year,raceRound,prix,driverFirstName,driverLastName,finishingPosition,positionOrder,time,constructor
0,1,1950,1,Great Britain,Nino,Farina,1,1,2:13:23.600,Alfa Romeo
23,2,1950,2,Monaco,Juan Manuel,Fangio,1,1,3:13:18.700,Alfa Romeo
78,4,1950,4,Switzerland,Nino,Farina,1,1,2:02:53.700,Alfa Romeo
96,5,1950,5,Belgium,Juan Manuel,Fangio,1,1,2:47:26.000,Alfa Romeo
110,6,1950,6,France,Juan Manuel,Fangio,1,1,2:57:52.800,Alfa Romeo


In [19]:
wins.info()

<class 'pandas.core.frame.DataFrame'>
Index: 528 entries, 0 to 22375
Data columns (total 10 columns):
raceId               528 non-null int64
year                 528 non-null int64
raceRound            528 non-null int64
prix                 528 non-null object
driverFirstName      528 non-null object
driverLastName       528 non-null object
finishingPosition    528 non-null object
positionOrder        528 non-null int64
time                 528 non-null object
constructor          528 non-null object
dtypes: int64(4), object(6)
memory usage: 45.4+ KB


In [20]:
wins.head(22)

,raceId,year,raceRound,prix,driverFirstName,driverLastName,finishingPosition,positionOrder,time,constructor
0,1,1950,1,Great Britain,Nino,Farina,1,1,2:13:23.600,Alfa Romeo
23,2,1950,2,Monaco,Juan Manuel,Fangio,1,1,3:13:18.700,Alfa Romeo
78,4,1950,4,Switzerland,Nino,Farina,1,1,2:02:53.700,Alfa Romeo
96,5,1950,5,Belgium,Juan Manuel,Fangio,1,1,2:47:26.000,Alfa Romeo
110,6,1950,6,France,Juan Manuel,Fangio,1,1,2:57:52.800,Alfa Romeo
130,7,1950,7,Italy,Nino,Farina,1,1,2:51:17.400,Alfa Romeo
158,8,1951,1,Switzerland,Juan Manuel,Fangio,1,1,2:07:53.640,Alfa Romeo
213,10,1951,3,Belgium,Nino,Farina,1,1,2:45:46.200,Alfa Romeo
226,11,1951,4,France,Juan Manuel,Fangio,1,1,3:22:11.000,Alfa Romeo
227,11,1951,4,France,Luigi,Fagioli,1,2,SHC,Alfa Romeo


One thing with the earlier F1 races that I've noted in my previous analysis attempts is that the there were a lot of shared drives. In this dataset, those rows are easier to find. In the column for time, they have a value of `SHC`. In doing my calculations, I'll ignore these rows.

In [21]:
wins_clean = wins[wins["time"] != "SHC"]

In [22]:
wins_clean.head(6)

,raceId,year,raceRound,prix,driverFirstName,driverLastName,finishingPosition,positionOrder,time,constructor
0,1,1950,1,Great Britain,Nino,Farina,1,1,2:13:23.600,Alfa Romeo
23,2,1950,2,Monaco,Juan Manuel,Fangio,1,1,3:13:18.700,Alfa Romeo
78,4,1950,4,Switzerland,Nino,Farina,1,1,2:02:53.700,Alfa Romeo
96,5,1950,5,Belgium,Juan Manuel,Fangio,1,1,2:47:26.000,Alfa Romeo
110,6,1950,6,France,Juan Manuel,Fangio,1,1,2:57:52.800,Alfa Romeo
130,7,1950,7,Italy,Nino,Farina,1,1,2:51:17.400,Alfa Romeo


In [23]:
wins_grouped = wins_clean.copy().groupby(["year", "constructor"])

In [24]:
count_wins = wins_grouped.finishingPosition.count()

In [25]:
count_wins.head(20)

year  constructor
1950  Alfa Romeo     6
1951  Alfa Romeo     4
1952  Ferrari        7
1953  Ferrari        7
1954  Mercedes       4
1955  Mercedes       5
1956  Ferrari        5
1957  Maserati       4
1958  Ferrari        2
1959  Cooper         5
1960  Cooper         6
1961  Ferrari        5
1962  BRM            4
1963  Lotus          7
1964  Ferrari        3
1965  Lotus          6
1966  Brabham        4
1967  Brabham        4
1968  Lotus          5
1969  Matra          6
Name: finishingPosition, dtype: int64

In [26]:
count_wins = count_wins.to_frame().reset_index()

In [27]:
count_wins.head(13)

,year,constructor,finishingPosition
0,1950,Alfa Romeo,6
1,1951,Alfa Romeo,4
2,1952,Ferrari,7
3,1953,Ferrari,7
4,1954,Mercedes,4
5,1955,Mercedes,5
6,1956,Ferrari,5
7,1957,Maserati,4
8,1958,Ferrari,2
9,1959,Cooper,5


In [28]:
count_wins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 3 columns):
year                 69 non-null int64
constructor          69 non-null object
finishingPosition    69 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.7+ KB


The dataFrame, `count_wins`, has the number of wins a team got in their championship run. Let's rename the last column from `positionOrder` to `wins`

In [29]:
count_wins = count_wins.rename(index=str, columns={"finishingPosition": "wins"})

In [30]:
count_wins.sort_values(by="wins",ascending=False)

,year,constructor,wins
66,2016,Mercedes,19
65,2015,Mercedes,16
64,2014,Mercedes,16
38,1988,McLaren,15
54,2004,Ferrari,15
52,2002,Ferrari,15
63,2013,Red Bull Racing,13
34,1984,McLaren,12
67,2017,Mercedes,12
61,2011,Red Bull Racing,12


Let's save the list:

In [31]:
count_wins.to_csv("../data/output/win_count.csv",index=False)

Looking at this list, Mercedes' 2016 run would be the best F1 season. But we're not accounting for the fact that the number of races per season has varied. Let's try to take that into account by finding the number of races in a season and dividing the number of wins by the number of races in the season.

In [32]:
normalize_wins = count_wins.copy()

In [33]:
def races_in_season(row):
    y = int(row["year"])
    season = results2[results2.year == y]
    number = season.raceRound.max()
    return number

In [34]:
normalize_wins["races"] = normalize_wins.apply(races_in_season, axis=1)

In [35]:
normalize_wins.head()

,year,constructor,wins,races
0,1950,Alfa Romeo,6,7
1,1951,Alfa Romeo,4,9
2,1952,Ferrari,7,8
3,1953,Ferrari,7,9
4,1954,Mercedes,4,9


Now we can find the percentage of races each team had in their season:

In [36]:
normalize_wins["win_percentage"] = (normalize_wins["wins"] / normalize_wins["races"])

In [37]:
normalize_wins.head()

,year,constructor,wins,races,win_percentage
0,1950,Alfa Romeo,6,7,0.857143
1,1951,Alfa Romeo,4,9,0.444444
2,1952,Ferrari,7,8,0.875000
3,1953,Ferrari,7,9,0.777778
4,1954,Mercedes,4,9,0.444444


And now we can resort from highest to lowest. I could scale things by 100 here to get percentages, but I can also do that when I get to plotting.

In [38]:
normalize_wins.sort_values(by="win_percentage",ascending=False)

,year,constructor,wins,races,win_percentage
38,1988,McLaren,15,16,0.937500
66,2016,Mercedes,19,21,0.904762
52,2002,Ferrari,15,17,0.882353
2,1952,Ferrari,7,8,0.875000
0,1950,Alfa Romeo,6,7,0.857143
65,2015,Mercedes,16,19,0.842105
64,2014,Mercedes,16,19,0.842105
54,2004,Ferrari,15,18,0.833333
3,1953,Ferrari,7,9,0.777778
46,1996,Williams,12,16,0.750000


From this we can see that McLaren's 1988 run was actually better than Mercedes' 2016 run — McLaren only lost 1 race. Let's save this dataFrame for later.

In [39]:
normalize_wins.to_csv("../data/output/win_count_normalized.csv", index=False)

Now that we have this figured out, we can move to doing things with the podiums.

---

## Idea 02: Podiums

Looking at the wins is a good start, but there are a lot of factors about the team's performance over a season that it leaves out.

* It only shows a very narrow slice of the team's drivers's performance. If we only know that one of the drivers won, we have no idea how the other driver did.
* It offers a limited amount of comparison. Winning is a binary variable — you win or you don't. When looking at the history of the sport, things are greyer. For example, Keke Rosberg won the driver's cup in 1982, but he only had one victory that season. Looking only at the number of wins doesn't provide any information about how this happened.

We can dig a little deeper and look at podiums. The podium refers to the drivers who finished first, second, and third in any given race. A team that consistenly has both drivers on the podium over a season is doing amazing. (ex: Mercedes in 2019 is dominating right now with Bottas and Hamilton in the podium for every race)

In [40]:
podiums = results2[results2.finishingPosition.isin(["1", "2", "3"])]

In [41]:
podiums.head(23)

,raceId,year,raceRound,prix,driverFirstName,driverLastName,finishingPosition,positionOrder,time,constructor
0,1,1950,1,Great Britain,Nino,Farina,1,1,2:13:23.600,Alfa Romeo
1,1,1950,1,Great Britain,Luigi,Fagioli,2,2,+2.600s,Alfa Romeo
2,1,1950,1,Great Britain,Reg,Parnell,3,3,+52.000s,Alfa Romeo
23,2,1950,2,Monaco,Juan Manuel,Fangio,1,1,3:13:18.700,Alfa Romeo
78,4,1950,4,Switzerland,Nino,Farina,1,1,2:02:53.700,Alfa Romeo
79,4,1950,4,Switzerland,Luigi,Fagioli,2,2,+0.400s,Alfa Romeo
96,5,1950,5,Belgium,Juan Manuel,Fangio,1,1,2:47:26.000,Alfa Romeo
97,5,1950,5,Belgium,Luigi,Fagioli,2,2,+14.000s,Alfa Romeo
110,6,1950,6,France,Juan Manuel,Fangio,1,1,2:57:52.800,Alfa Romeo
111,6,1950,6,France,Luigi,Fagioli,2,2,+25.700s,Alfa Romeo


As I mentioned earlier, shared drives were a common thing in the early years of Formula1 and they're findable in the data by looking for any rows for where the `time` is `SHC`. 

When looking at the podiums, I'll be doing the same. 

In [42]:
podiums_clean = podiums[podiums.time != "SHC"]

In [43]:
podiums_clean.head(21)

,raceId,year,raceRound,prix,driverFirstName,driverLastName,finishingPosition,positionOrder,time,constructor
0,1,1950,1,Great Britain,Nino,Farina,1,1,2:13:23.600,Alfa Romeo
1,1,1950,1,Great Britain,Luigi,Fagioli,2,2,+2.600s,Alfa Romeo
2,1,1950,1,Great Britain,Reg,Parnell,3,3,+52.000s,Alfa Romeo
23,2,1950,2,Monaco,Juan Manuel,Fangio,1,1,3:13:18.700,Alfa Romeo
78,4,1950,4,Switzerland,Nino,Farina,1,1,2:02:53.700,Alfa Romeo
79,4,1950,4,Switzerland,Luigi,Fagioli,2,2,+0.400s,Alfa Romeo
96,5,1950,5,Belgium,Juan Manuel,Fangio,1,1,2:47:26.000,Alfa Romeo
97,5,1950,5,Belgium,Luigi,Fagioli,2,2,+14.000s,Alfa Romeo
110,6,1950,6,France,Juan Manuel,Fangio,1,1,2:57:52.800,Alfa Romeo
111,6,1950,6,France,Luigi,Fagioli,2,2,+25.700s,Alfa Romeo


Now let's count them.

In [44]:
count_podiums = podiums_clean.groupby(["year", "constructor"]).finishingPosition.count()

In [45]:
count_podiums.head()

year  constructor
1950  Alfa Romeo     12
1951  Alfa Romeo      9
1952  Ferrari        17
1953  Ferrari        16
1954  Mercedes        7
Name: finishingPosition, dtype: int64

In [46]:
count_podiums = count_podiums.to_frame().reset_index()

In [47]:
count_podiums.head()

,year,constructor,finishingPosition
0,1950,Alfa Romeo,12
1,1951,Alfa Romeo,9
2,1952,Ferrari,17
3,1953,Ferrari,16
4,1954,Mercedes,7


In [48]:
count_podiums.sort_values(by="finishingPosition",ascending=False).head(20)

,year,constructor,finishingPosition
66,2016,Mercedes,33
65,2015,Mercedes,32
64,2014,Mercedes,31
54,2004,Ferrari,29
52,2002,Ferrari,27
61,2011,Red Bull Racing,27
67,2017,Mercedes,26
38,1988,McLaren,25
68,2018,Mercedes,25
51,2001,Ferrari,24


We can save this version of raw podium counts and get to normalizing the podiums over the season to better compare the different runs. But before that let's rename the column we used to count things.

In [49]:
count_podiums = count_podiums.rename(index=str, columns={"finishingPosition": "podiums"})

In [50]:
count_podiums.to_csv("../data/output/podium_count.csv")

To normalize the podiums, we'll need to get the number of possible podiums in a season. That's the number of races times 3. When counting the number of wins we figured out how to do the races already.

In [51]:
normalize_podiums = count_podiums.copy()

In [52]:
normalize_podiums["races"] = normalize_podiums.apply(races_in_season, axis=1)

In [53]:
normalize_podiums.head()

,year,constructor,podiums,races
0,1950,Alfa Romeo,12,7
1,1951,Alfa Romeo,9,9
2,1952,Ferrari,17,8
3,1953,Ferrari,16,9
4,1954,Mercedes,7,9


While there are three podium spots every race, for the majority of Formula 1 seasons, teams have only had two drivers, so the maximum number of podium spots a team can have in a season is `2 * (number of races)`

In [54]:
def podiums_in_season(row):
    return row.races * 2

In [55]:
normalize_podiums["podium_spots"] = normalize_podiums.apply(podiums_in_season, axis=1)

In [56]:
normalize_podiums.head()

,year,constructor,podiums,races,podium_spots
0,1950,Alfa Romeo,12,7,14
1,1951,Alfa Romeo,9,9,18
2,1952,Ferrari,17,8,16
3,1953,Ferrari,16,9,18
4,1954,Mercedes,7,9,18


And now we have everything to compute a podium percentage.

In [57]:
normalize_podiums["podium_percentage"] = normalize_podiums.podiums / normalize_podiums.podium_spots

In [58]:
normalize_podiums.head(20)

,year,constructor,podiums,races,podium_spots,podium_percentage
0,1950,Alfa Romeo,12,7,14,0.857143
1,1951,Alfa Romeo,9,9,18,0.500000
2,1952,Ferrari,17,8,16,1.062500
3,1953,Ferrari,16,9,18,0.888889
4,1954,Mercedes,7,9,18,0.388889
5,1955,Mercedes,10,7,14,0.714286
6,1956,Ferrari,10,9,18,0.555556
7,1957,Maserati,10,9,18,0.555556
8,1958,Ferrari,14,11,22,0.636364
9,1959,Cooper,13,9,18,0.722222


In [59]:
normalize_podiums.sort_values(by="podium_percentage",ascending=False).head(20)

,year,constructor,podiums,races,podium_spots,podium_percentage
2,1952,Ferrari,17,8,16,1.062500
11,1961,Ferrari,14,7,14,1.000000
3,1953,Ferrari,16,9,18,0.888889
0,1950,Alfa Romeo,12,7,14,0.857143
65,2015,Mercedes,32,19,38,0.842105
64,2014,Mercedes,31,19,38,0.815789
54,2004,Ferrari,29,18,36,0.805556
52,2002,Ferrari,27,17,34,0.794118
66,2016,Mercedes,33,21,42,0.785714
38,1988,McLaren,25,16,32,0.781250


The top four teams all had more than two drivers in their runs. What if we look at increasing the podium spots to match the fact that a team could have 3 drivers.

In [60]:
def podiums_in_season2(row):
    return row.races * 3

In [64]:
normalize_podiums["podium_spots2"] = normalize_podiums.apply(podiums_in_season2, axis=1)

In [65]:
normalize_podiums["podium_percentage2"] = normalize_podiums.podiums / normalize_podiums.podium_spots2

In [66]:
normalize_podiums.sort_values(by="podium_percentage2",ascending=False).head(20)

,year,constructor,podiums,races,podium_spots,podium_percentage,podium_spots2,podium_percentage2
2,1952,Ferrari,17,8,16,1.062500,24,0.708333
11,1961,Ferrari,14,7,14,1.000000,21,0.666667
3,1953,Ferrari,16,9,18,0.888889,27,0.592593
0,1950,Alfa Romeo,12,7,14,0.857143,21,0.571429
65,2015,Mercedes,32,19,38,0.842105,57,0.561404
64,2014,Mercedes,31,19,38,0.815789,57,0.543860
54,2004,Ferrari,29,18,36,0.805556,54,0.537037
52,2002,Ferrari,27,17,34,0.794118,51,0.529412
66,2016,Mercedes,33,21,42,0.785714,63,0.523810
38,1988,McLaren,25,16,32,0.781250,48,0.520833


I guess it doesn't change things. But maybe I should account for the number of drivers in each season when I account for podiums. There are `3 * number of races` podium spots over a given season, But for most of the seasons, only two drivers so at most a team can only get `2 * number of races`. **Maybe we can divide the `podium_percentage` by the number of drivers a team had? That would create a podiums per season per driver and could help with comparisons?** I will come back to this later.